I came across this problem on [HackerRank](https://www.hackerrank.com/challenges/missing-stock-prices "Missing Stock Prices Challenge"). I'm no expert on either data analysis or finance. But when processing historical financial data, it is common to have missing data points for reasons such as: 
* Incorrect backfilled data (link to Bloomberg article coming soon)
* Poor record keeping
* Stock exchanges have different holiday schedules. So it becomes an obstacle to perform a time series analysis on securities traded on different exchanges. 

So sometimes (not always) it makes sense to calculate and predict historical prices.

The problem on HackerRank provides a dataset of a stock's highest price each day, with a few points missing. The first line contains the number of rows. Each line contains a date and the highest price that day.

250

1/3/2012 16:00:00	26.96

1/4/2012 16:00:00	27.47

1/5/2012 16:00:00	27.728

...

12/27/2012 16:00:00	27.09

12/28/2012 16:00:00	26.9

12/31/2012 16:00:00	26.77

### Let's start with parsing the input. 

We don't actually need to use the date in this case since there is only one data point on each trading day. The interval of each price is always exactly one trading day. 

In [32]:
import fileinput

file = fileinput.FileInput('input')

# This is the number of rows
n = int(file.readline())

raw_prices = []
for i in range(0, n):
    date, price = file.readline().split('\t')
    raw_prices.append(price)

Now, ```raw_prices``` contains the raw input of prices. We want to distinguish valid prices from the missing ones. Also, the indices (x-axis) will be the independent variable, and the prices (y-axis) will be the dependent variable - target features. 

In [41]:
# x-axis: date or the index of each data point
x = [] # Indices of valid prices
x_missing = [] # Indices of missing data

# y-axis: prices
y = []

for i in range(0, n):
    if 'Missing' in raw_prices[i]:
        x_missing.append(i)
    else:
        # Valid price
        x.append(i)
        y.append(float(raw_prices[i]))

At the beginning, I calculated the prices based on 5-day averages. But it wasn't very accurate. So I will try interpolation. We will use a simple [scipy](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.interpolate.UnivariateSpline.html#scipy.interpolate.UnivariateSpline) Univariate Spline for interpolation. We also need [numpy](www.numpy.org) to wrap a list for scipy to process. 

### Interpolation

Interpolation works quite well for data points in between existing points. In our case, missing historical stock prices fall perfectly into this category. On the other hand, extrapolation is used for data points **outside** of a given dataset. e.g. stock prices **tomorrow** (extrapolation won't work so well though). [This](https://www.quora.com/What-is-the-difference-between-interpolation-and-extrapolation) is a quick but fantastic explanation that I found helpful.

UnivariateSpline simply means using spline interpolation wiht one-dimensional data points. [Spline](https://en.wikipedia.org/wiki/Spline_(mathematics%29) is basically a bunch of piecewise polynomial functions that fit a given dataset. Later on, we can use it to *interpolate* data points. The ```s``` parameter passed to the Spline object is the smoothness factor. 

In [50]:
from scipy.interpolate import UnivariateSpline
import numpy as np

spline = UnivariateSpline(np.array(x), np.array(y), s=2)

# We start interpolation here
for i in x_missing:
    print(spline(i))

32.56701749439569
31.98466008638443
32.588716107733376
29.401547180365338
28.97326520862967
28.68606129302796
30.523689377232575
29.885189408695812
29.582872200153187
30.867367120670803
31.15581794687907
31.54592188188132
29.668506599317897
29.41441268832914
29.488051778620097
28.925535432472056
29.840927304489984
28.111090022235476
26.845411394867742
27.378677906828305


The smoothness of a spline function is defined as the number of derivatives that it has that are continuous. A knot is where two piecewise functions are connected. Stock price movements are usually inconsistent. The spline function wouldn't be so smooth with a large number of knots. The 'smoothness factor' dictates how many knots there should be. The more knots there are, the less smooth the function can be overall since knots may be discontinuous. The ```s``` argument refers to how much the residual sum of squares of each existing data point should be minimized. Smaller RSS yields a function that is less smooth. 2 seems to work quite well with the datasets in this case. 

Thanks for reading! This is a very simple analysis and manipulation on the dataset. I will play around with this dataset a bit more, possibly with Support Vector Machine!